<a href="https://colab.research.google.com/github/NoufAlmufadi/-E-DFu-Net/blob/main/E_DFu_Net_for_ischemia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow.keras
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
from keras import optimizers
from keras.layers import Input, Lambda, Dense, Flatten, Dropout
from keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import metrics
from glob import glob
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.svm import SVC
from sklearn import metrics
import os
from imutils import paths
from cv2 import imread
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
%tensorflow_version 2.x

In [ ]:
pip install -q -U keras-tuner

In [ ]:
pip install plot_keras_history

In [ ]:
from plot_keras_history import show_history, plot_history


In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import glob
import tensorflow as tf
from sklearn import preprocessing



train_images = []
train_labels = []

test_images = []
test_labels = []

val_images = []

for directory_path in glob.glob("/content/drive/My Drive/datasets/ischaemia_12000agusplit/train/*"):
    label = os.path.split(directory_path)
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path)
        image=tf.keras.preprocessing.image.load_img(img_path, color_mode='rgb',
        target_size= (224,224))
        image=np.array(image)
        train_images.append(image)
        train_labels.append(label)

for directory_path1 in glob.glob("/content/drive/My Drive/datasets/ischaemia_12000agusplit/test/*"):
    label1 = os.path.split(directory_path1)
    print(label1)
    for img_path1 in glob.glob(os.path.join(directory_path1, "*.jpg")):
        print(img_path1)
        image1=tf.keras.preprocessing.image.load_img(img_path1, color_mode='rgb',
        target_size= (224,224))
        image1=np.array(image1)
        test_images.append(image1)
        test_labels.append(label1)

for directory_path in glob.glob("/content/drive/My Drive/datasets/ischaemia_12000agusplit/val/*"):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path)
        image=tf.keras.preprocessing.image.load_img(img_path, color_mode='rgb',
        target_size= (224,224))
        image=np.array(image)
        val_images.append(image)



train_images = np.array(train_images)
train_labels = np.array(train_labels)


test_images = np.array(test_images)
test_labels = np.array(test_labels)

val_images = np.array(val_images)

print(test_labels.shape)
print(train_labels.shape)

print(train_images.shape)
print(test_images.shape)
print(val_images.shape)


In [ ]:
from tensorflow.keras.applications import EfficientNetB0

EfficientNetB0 = EfficientNetB0(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [ ]:
EfficientNetB0.input


In [ ]:
for layer in EfficientNetB0.layers:
  layer.trainable = False

In [ ]:
from tensorflow.keras.layers import BatchNormalization

x = Flatten()(EfficientNetB0.output)
x = tf.keras.layers.Dropout(0.5)(x)
x=BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001)(x)
x = tf.keras.layers.Dropout(0.5)(x)
x=BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001)(x)
x=Dense(256, kernel_regularizer= tensorflow.keras.regularizers.l2(l= 0.016), activity_regularizer= tensorflow.keras.regularizers.l1(0.006),bias_regularizer= tensorflow.keras.regularizers.l1(0.006), activation= 'relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x=Dense(128, kernel_regularizer= tensorflow.keras.regularizers.l2(l= 0.016), activity_regularizer= tensorflow.keras.regularizers.l1(0.006),bias_regularizer= tensorflow.keras.regularizers.l1(0.006), activation= 'relu')(x)
x=Dropout(rate= 0.45, seed= 123)(x)
prediction = Dense(2,activation='sigmoid')(x)
model = Model(inputs=EfficientNetB0.input, outputs=prediction)

In [ ]:
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()

In [ ]:
train_set = train_datagen.flow_from_directory('/content/drive/My Drive/datasets/ischaemia_12000agusplit/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

In [ ]:
# Create a generator for the test data without augmentation
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/datasets/ischaemia_12000agusplit/test',
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='categorical' )

In [ ]:
val_set = val_datagen.flow_from_directory('/content/drive/My Drive/datasets/ischaemia_12000agusplit/val',
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='categorical' )

In [ ]:
from keras.optimizers import Adamax
model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss',verbose=1, patience=20)

In [ ]:

import time

#starting time
start = time.time()

batch_size = 32
#batch_size = 64
#batch_size = 128
hist = model.fit(train_set, steps_per_epoch = train_set.samples//batch_size,
                 validation_data = val_set,validation_steps = val_set.samples//batch_size,
                 epochs = 100,callbacks=[es]
                 )
# end time
end = time.time()

# total time taken
print("Execution time of the program is- ", end-start)

In [ ]:
# plot the loss
plt.plot(hist.history['loss'], label='train loss',marker='o')
plt.plot(hist.history['val_loss'], label='val loss',marker='o')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:

# plot the loss
plt.plot(hist.history['accuracy'], label='train accuracy',marker='o')
plt.plot(hist.history['val_accuracy'], label='val accuracy',marker='o')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_labels = train_labels[:, 1]
test_labels = test_labels[:, 1]

In [ ]:

label_mapping = {'Aug-Negative': 0, 'Aug-Positive': 1}
train_labels = np.array([label_mapping[label] for label in train_labels])
test_labels = np.array([label_mapping[label] for label in test_labels])

In [ ]:
from sklearn import metrics

# Make predictions on the training data
train_pred_probs = model.predict(train_images)  # Use the original training data
train_pred_labels = np.argmax(train_pred_probs, axis=1)

# Make predictions on the test data
test_pred_probs = model.predict(test_images)
test_pred_labels = np.argmax(test_pred_probs, axis=1)

# Ensure that train_labels and test_labels are correctly shaped
# They should be one-dimensional arrays if not already
train_labels = np.squeeze(train_labels)
test_labels = np.squeeze(test_labels)

# Calculate the accuracy scores
train_accuracy = metrics.accuracy_score(train_labels, train_pred_labels)
test_accuracy = metrics.accuracy_score(test_labels, test_pred_labels)

print("\nTraining Accuracy Score:", train_accuracy * 100)
print("\nTesting Accuracy Score:", test_accuracy * 100)

In [ ]:
feat_train=model.predict(train_images)
feat_train=feat_train.reshape(feat_train.shape[0], -1)

feat_test=model.predict(test_images)
feat_test=feat_test.reshape(feat_test.shape[0], -1)

In [ ]:
from sklearn.metrics import precision_score

from sklearn.metrics import recall_score

from sklearn.metrics import f1_score

from sklearn.metrics import cohen_kappa_score

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score





# accuracy: (tp + tn) / (p + n)

accuracy = accuracy_score(test_labels, test_pred_labels)

print('Accuracy: %f' % accuracy)

# precision tp / (tp + fp)
precision = precision_score(test_labels, test_pred_labels)

print('Precision: %f' %precision)

# recall: tp / (tp + fn)

recall = recall_score(test_labels, test_pred_labels)

print('Recall: %f' % recall)


Specificity = recall_score(test_labels, test_pred_labels, pos_label= 0)

print('Specificity: %f' % Specificity)

# f1: 2 tp / (2 tp + fp + fn)

f1 = f1_score(test_labels, test_pred_labels)

print('F1 score: %f' % f1)

roc_auc = roc_auc_score(test_labels, test_pred_labels)

print('roc_auc: %f' % roc_auc)